<a href="https://colab.research.google.com/github/JasmineTsai1995/llama_club/blob/main/R2_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import json
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
import re

In [29]:
!pip install langchain

In [2]:
!pip install transformers torch langchain

In [2]:
!pip install langchain transformers torch accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 20.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [3]:
!pip install accelerate

In [4]:
!pip install -Uqqq


!pip -qqq install bitsandbytes accelerate

ERROR: You must give at least one requirement to install (see "pip help install")


In [5]:
!pip install langchain transformers accelerate bitsandbytes

In [ ]:
#!pip install --upgrade transformers torch

##**模型**

In [5]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

In [6]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.0001,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.15
)

In [10]:
output = hf_pipeline("Hello, how are you?")
print(output)

[{'generated_text': "Hello, how are you? I'm doing well. How can I help you today?"}]


In [37]:
news_1 ="""
嘉義縣水上鄉三界村一處廢棄豬舍鐵皮屋昨深夜11點起火，地點位於台82線快速道路旁，濃煙竄天嚇壞許多民眾。警消獲報到場滅火，疑堆放大量塑膠製品迅速延燒，並傳出一名老人失聯，經徹夜搜索，消防人員今早9點多在旁民宅找到驚魂未定70歲男子，將他送醫。
據悉，水上鄉三界埔一處豬舍昨夜失火，現場疑堆放大量廢棄塑膠製品，導致火勢延燒迅速猛烈，鐵皮屋被燒到變形倒塌，火滅後仍持續悶燒，消防局出動20車、47人搶救，並出動大型機具開挖滅火，經徹夜9小時灌救滅火，初判燃燒面積約為2000平方公尺。
一名老翁平時住在廢棄豬舍附近，昨夜傳出疑似失聯，消防人員徹夜擴大搜索火場，並未在廢棄豬舍找到他，直到今天早上9點48分才在火場後方建築物2樓逃生梯發現，幸未被火勢延燒波及，只有建築物表面遭濃煙熏黑，老翁驚魂未定，但未受傷。
嘉義縣消防局表示，年約70歲男子待在後方建築物2樓逃生梯，意識清楚、無明顯外傷，給予救護處置後，由救護車預防性送往聖馬醫院診治。提醒民眾遇到火災切勿慌張，「小火快逃、濃煙關門」是火場逃生不二法門，且隨時注意逃生路徑，就能提高獲救機會。
"""
news_2 ="""
台南市成功大學勝利校區體育館今天晚上9點多發生火災，消防局接獲報案派遣13車27人前往搶救，現場為2樓健身房的天花板管線冒煙，人員都已進行疏散，消防人員出水滅火，火勢約半小時後撲滅。
消防人員說，可能是體育館空調設備故障引起火災；台南市消防局說，起火真正原因需由消災調查科人員進一步鑑定查明。
台南市消防局救災救護指揮中心今晚9點32分接獲報案，成功大學勝利校區體育館發生火災，中心派遣東門等消防分隊13車27人前往搶救，消防人員到達現場回報為2樓健身房的天花板管線冒煙，已進行疏散。義消到達外觀有濃煙，校方有初期滅火。
據了解，2樓為健身房使用，無延燒之虞，館內人員皆撤出。消防人員出水降溫，及排煙。火勢於晚上10點3分撲滅。
"""
news_3 ="""
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
"""

In [40]:
def extract_fire_info(news_text):
    question = f"請將提供的火災新聞描述中只要萃取出以下四點「起火點」、「起火原因」、「起火時間」、「是否有人死亡」，舉例：起火點: 廚房,起火原因: 電線起火, 起火時間: 早上9點, 死亡與否: false，並以 JSON 格式回傳，並轉成 Python Dict。{news_text}"
    response = hf_pipeline(question)[0]['generated_text']
    print (response)
    # 提取 JSON 部份
    start_idx = response.find("{")
    end_idx = response.rfind("}") + 1
    json_str = response[start_idx:end_idx]
    result_dict = json.loads(json_str)

    return result_dict

result = extract_fire_info(news_3) # 換新聞

print("Output:", result)

請將提供的火災新聞描述中只要萃取出以下四點「起火點」、「起火原因」、「起火時間」、「是否有人死亡」，舉例：起火點: 廚房,起火原因: 電線起火, 起火時間: 早上9點, 死亡與否: false，並以 JSON 格式回傳，並轉成 Python Dict。
超衰！50歲陳姓男子在高雄鳳山租屋，未料某天睡著時，床頭夾式檯燈電源線短路起火，火勢一發不可收拾，一路延燒到至少6處樓層，雖未釀成傷亡，但經消防火調科調查，證據都指向陳男釀災可能性最大，被5名租客聯合提告，法院審理後，將他依法判4個月有期徒刑，得易科罰金。
判決指出，50歲陳男高雄鳳山租屋，2023年初凌晨睡覺時，臥室床頭夾式檯燈電源線短路起火，瞬間燃燒周邊木質家具及電器，火勢一發不可收拾，一路延燒到至少6處樓層，造成建物客廳、廚房、廁所、床墊、家具等物被燒毀。
消防撲滅火勢後，雖未釀成人員傷亡，但受害5名房客聯合對陳男提告；經火調科調查，火災起火點位在陳男住處臥室靠客廳走廊床頭櫃附近，起火原因以電氣因素引燃可能性最大，他因此吃上官司。
陳男警詢及偵訊時，都坦承此事，被檢方依失火燒燬他人之物等罪嫌起訴，他也在審理過程中自白。
法官認為，陳男使用電器用品時，未注意夾式檯燈電源線使用狀況，導致火災發生，除燒燬自己租用住宅外，還波及隔鄰其他住戶，造成多人損失，考量坦承犯行，態度尚可，審理後，將他依犯失火燒燬現供人使用之住宅罪，處4個月有期徒刑，得易科罰金。
{
"fire_point": "臥室",
"cause": "插頭電線短路",
"time": "凌晨",
"death": false
}
Output: {'fire_point': '臥室', 'cause': '插頭電線短路', 'time': '凌晨', 'death': False}




---


### 作業end

### 以下不是作業

### 以下不是作業

### 以下不是作業

In [ ]:
news = """
南韓京畿道鋰亞電池（Aricell）工廠，今（24）日上午驚傳大規模火災。台灣時間下午三點，消防人員在搜索工廠內部時、發現逾20具焦黑遺體，傍晚近六點，韓媒報導，死者增加至22人，其中18人為中國籍、1名寮國籍，2死者韓國籍。據了解，工廠當時有67人在工作，目前仍有1人失聯。
綜合《MBC》等韓媒報導，消防人員於當地時間上午10點31分接獲通報，稱京畿道華城市西新面一座鋰電池工廠發生火災。工廠總共有3樓，為鋼筋混凝土結構、總面積達2300平方公尺。由於工廠2樓保管著3.5萬個鋰電池，疑為1顆電池起火後急遽擴散，引發連續爆炸。當地消防總計派出145名人員與50輛滅火車輛到場。
建築物竄出烈焰、伴隨濃煙不斷傳來爆炸聲響。消防人員發現1名沒有心跳的60多歲男子，送醫後宣告死亡。另有2人重傷、4人輕傷。台灣時間下午三點左右，韓媒報導消防人員在工廠內部尋獲逾20具遺體。據了解，火災發生當時有67名員工在建築內。
消防人員指出，電池相關火災不易滅火，目前仍難以進入工廠內部搜救，致力於防止火勢延伸到其他工廠。南韓總統尹錫悅指示消防人員全力救災。但考慮到火勢急遽燃燒，呼籲消防人員要做好安全對策。
"""

In [ ]:
news1 = """
2024/06/25
台中市新社區著名的景點之一的「安妮公主花園」今(25)日傍晚發生火警，主要木造二樓餐廳遭大火吞噬，消防局出動轄區五個單位前往救援，火勢在晚間18時50分撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
台中市消防局今(25)日18:09接獲民眾報案指出，在新社區中興街有火警，立即派遣第二大隊、轄區新社等5個單位，出動各式消防車12輛、救護車1輛，人員34名，由組長林諺樟擔任現場指揮官。現場為2層木造建築物，消防人員佈線搶救，火勢於18:37分控制，18:50撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
一名林姓民眾表示，傍晚要回家時就發現木屋冒出濃煙有火光出現，就立即打電話報警求援。「安妮公主花園」占地約三公頃，木屋在2002年建造是新社著名的打卡景點，園區內有復育螢火蟲、鍬形蟲等，種植多樣式的花朵，木屋內部設有餐廳有多種的餐點料理，是許多年輕人喜歡聚會賞景的地方。
"""

In [ ]:
question = f"請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」並以 JSON 格式回傳：{news}"

In [ ]:
# 生成答案
response = hf_pipeline(question)[0]['generated_text']
print("生成的原始答案：", response)

生成的原始答案： 請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」並以 JSON 格式回傳：
南韓京畿道鋰亞電池（Aricell）工廠，今（24）日上午驚傳大規模火災。台灣時間下午三點，消防人員在搜索工廠內部時、發現逾20具焦黑遺體，傍晚近六點，韓媒報導，死者增加至22人，其中18人為中國籍、1名寮國籍，2死者韓國籍。據了解，工廠當時有67人在工作，目前仍有1人失聯。
綜合《MBC》等韓媒報導，消防人員於當地時間上午10點31分接獲通報，稱京畿道華城市西新面一座鋰電池工廠發生火災。工廠總共有3樓，為鋼筋混凝土結構、總面積達2300平方公尺。由於工廠2樓保管著3.5萬個鋰電池，疑為1顆電池起火後急遽擴散，引發連續爆炸。當地消防總計派出145名人員與50輛滅火車輛到場。
建築物竄出烈焰、伴隨濃煙不斷傳來爆炸聲響。消防人員發現1名沒有心跳的60多歲男子，送醫後宣告死亡。另有2人重傷、4人輕傷。台灣時間下午三點左右，韓媒報導消防人員在工廠內部尋獲逾20具遺體。據了解，火災發生當時有67名員工在建築內。
消防人員指出，電池相關火災不易滅火，目前仍難以進入工廠內部搜救，致力於防止火勢延伸到其他工廠。南韓總統尹錫悅指示消防人員全力救災。但考慮到火勢急遽燃燒，呼籲消防人員要做好安全對策。
{
"fire_point": "南韓京畿道鋰亞電池（Aricell）工廠",
"fire_cause": "1顆鋰電池起火後遽增擴散",
"fire_time": "24日上午10點31分",
"deaths": {
    "total": 22,
    "nationality": {
        "Chinese": 18,
        "Myanmar": 1,
        "Korea": 2
    }
}


In [ ]:
question = f"請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」並以 JSON 格式回傳：{news1}"

In [ ]:
# First Try
response = hf_pipeline(question)[0]['generated_text']
print("生成的原始答案：", response)

生成的原始答案： 請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」並以 JSON 格式回傳：
2024/06/25
台中市新社區著名的景點之一的「安妮公主花園」今(25)日傍晚發生火警，主要木造二樓餐廳遭大火吞噬，消防局出動轄區五個單位前往救援，火勢在晚間18時50分撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
台中市消防局今(25)日18:09接獲民眾報案指出，在新社區中興街有火警，立即派遣第二大隊、轄區新社等5個單位，出動各式消防車12輛、救護車1輛，人員34名，由組長林諺樟擔任現場指揮官。現場為2層木造建築物，消防人員佈線搶救，火勢於18:37分控制，18:50撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
一名林姓民眾表示，傍晚要回家時就發現木屋冒出濃煙有火光出現，就立即打電話報警求援。「安妮公主花園」占地約三公頃，木屋在2002年建造是新社著名的打卡景點，園區內有復育螢火蟲、鍬形蟲等，種植多樣式的花朵，木屋內部設有餐廳有多種的餐點料理，是許多年輕人喜歡聚會賞景的地方。
{
"fire_point": "台中市新社區新興街",
"fire_time": "2024/06/25",
"fire_reason": "火災原因調查中",
"people_death": false,
"restaurant_name": "安妮公主花園"
}


In [ ]:
question = f"請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」。重要：請以完整 JSON 格式回傳。：{news1}"

In [ ]:
# Second Try
response = hf_pipeline(question)[0]['generated_text']
print("生成的原始答案：", response)

生成的原始答案： 請從以下新聞描述中萃取出「起火點」、「起火原因」、「起火時間」、「是否有人死亡」。重要：請以完整 JSON 格式回傳。：
2024/06/25
台中市新社區著名的景點之一的「安妮公主花園」今(25)日傍晚發生火警，主要木造二樓餐廳遭大火吞噬，消防局出動轄區五個單位前往救援，火勢在晚間18時50分撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
台中市消防局今(25)日18:09接獲民眾報案指出，在新社區中興街有火警，立即派遣第二大隊、轄區新社等5個單位，出動各式消防車12輛、救護車1輛，人員34名，由組長林諺樟擔任現場指揮官。現場為2層木造建築物，消防人員佈線搶救，火勢於18:37分控制，18:50撲滅，餐廳桌椅起火燃燒，無人員受傷受困，火災原因調查中。
一名林姓民眾表示，傍晚要回家時就發現木屋冒出濃煙有火光出現，就立即打電話報警求援。「安妮公主花園」占地約三公頃，木屋在2002年建造是新社著名的打卡景點，園區內有復育螢火蟲、鍬形蟲等，種植多樣式的花朵，木屋內部設有餐廳有多種的餐點料理，是許多年輕人喜歡聚會賞景的地方。
{
"fire_point": "台中市新社區著名的景點之一的『安妮公主花園』",
"fire_time": "2024/06/25",
"fire_cause": "火災原因調查中",
"people_death": "否"
}


In [ ]:
# 提取 JSON 字串的開始和結束位置
json_start = response.find('{')
json_end = response.rfind('}') + 1
json_response = response[json_start:json_end]

# 手動檢查和修復 JSON 字串
json_response = json_response.replace('\n', '').replace('，', ',').replace('：', ':')
json_response = json_response.replace('（', '(').replace('）', ')')

# 使用正則表達式自動移除多餘的逗號
json_response = re.sub(r',\s*}', '}', json_response)
json_response = re.sub(r',\s*]', ']', json_response)

# 檢查並補全缺失的閉合括號
stack = []
for i, char in enumerate(json_response):
    if char == '{':
        stack.append(i)
    elif char == '}':
        if stack:
            stack.pop()
        else:
            # 多餘的閉合括號，移除它
            json_response = json_response[:i] + json_response[i+1:]

# 如果還有未匹配的開括號，補齊閉合括號
while stack:
    json_response += '}'
    stack.pop()

# 將修復後的 JSON 字串轉換為 Python 字典
python_dict = json.loads(json_response)
print(python_dict)

{'fire_point': '南韓京畿道鋰亞電池(Aricell)工廠', 'fire_cause': '1顆鋰電池起火後遽增擴散', 'fire_time': '24日上午10點31分', 'deaths': {'total': 22, 'nationality': {'Chinese': 18, 'Myanmar': 1, 'Korea': 2}}}


In [ ]:
# 提取 JSON 字串的開始和結束位置
json_start = response.find('{')
json_end = response.rfind('}') + 1
json_response = response[json_start:json_end]

# 手動檢查和修復 JSON 字串
json_response = json_response.replace('\n', '').replace('，', ',').replace('：', ':')
json_response = json_response.replace('（', '(').replace('）', ')')

# 使用正則表達式自動移除多餘的逗號
json_response = re.sub(r',\s*}', '}', json_response)
json_response = re.sub(r',\s*]', ']', json_response)

# 檢查並補全缺失的閉合括號
stack = []
for i, char in enumerate(json_response):
    if char == '{':
        stack.append(i)
    elif char == '}':
        if stack:
            stack.pop()
        else:
            # 多餘的閉合括號，移除它
            json_response = json_response[:i] + json_response[i+1:]

# 如果還有未匹配的開括號，補齊閉合括號
while stack:
    json_response += '}'
    stack.pop()

# 將修復後的 JSON 字串轉換為 Python 字典
python_dict = json.loads(json_response)
print(python_dict)

{'fire_point': '台中市新社區新興街', 'fire_cause': '火災原因調查中', 'fire_time': '18:09', 'person_death': False}


In [ ]:
# 提取 JSON 字串的開始和結束位置
json_start = response.find('{')
json_end = response.rfind('}') + 1
json_response = response[json_start:json_end]

# 手動檢查和修復 JSON 字串
json_response = json_response.replace('\n', '').replace('，', ',').replace('：', ':')
json_response = json_response.replace('（', '(').replace('）', ')')

# 使用正則表達式自動移除多餘的逗號
json_response = re.sub(r',\s*}', '}', json_response)
json_response = re.sub(r',\s*]', ']', json_response)

# 檢查並補全缺失的閉合括號
stack = []
for i, char in enumerate(json_response):
    if char == '{':
        stack.append(i)
    elif char == '}':
        if stack:
            stack.pop()
        else:
            # 多餘的閉合括號，移除它
            json_response = json_response[:i] + json_response[i+1:]

# 如果還有未匹配的開括號，補齊閉合括號
while stack:
    json_response += '}'
    stack.pop()

# 將修復後的 JSON 字串轉換為 Python 字典
python_dict = json.loads(json_response)
print(python_dict)

{'fire_point': '台中市新社區新興街', 'fire_time': '2024/06/25', 'fire_reason': '火災原因調查中', 'people_death': False, 'restaurant_name': '安妮公主花園'}
